# Compare spectra produced by IMAGER and CASA

This notebook compares the spectra extracted from the data cubes produced by IMAGER and CASA. 

## Initialization

In [ ]:
!pip install -r requirements.txt > /dev/null

In [ ]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from spectral_cube import SpectralCube
from matplotlib import pyplot as plt
from compass import datadir, erda, extract_spectrum_from_cube

In [ ]:
%matplotlib widget

## Define the position, the frequency setting and the spectral window

In [ ]:
pos = SkyCoord("12h01m36.47554s", "-65d08m49.3509s", frame="fk5")
setting = 7
setting_casa = "g"
spw = 31

## Read the cube produced by CASA and extract spectrum from it

In [ ]:
if erda:
    cube_casa = SpectralCube.read(
        "../COMPASS/Reduced/BHR71_{}/Phase04/oussid.s10_0.BHR71-IRS1_sci.spw{}.cube.I.iter2.image.pbcor.fits".format(
            setting, spw
        )
    )
    restfreq_casa = cube_casa.header["RESTFRQ"] * u.Hz
    spectrum_casa = extract_spectrum_from_cube(cube_casa, pos)

## Read the cube produced by IMAGER and extract a spectrum

In [ ]:
cube_imager = SpectralCube.read(
    datadir + "bhr71/cubes/bhr71-set{}-spw{}-lines.fits".format(setting, spw)
)
restfreq_imager = cube_imager.header["RESTFRQ"] * u.Hz
spectrum_imager = extract_spectrum_from_cube(cube_imager, pos)

## Plot the spectra

In [ ]:
fig, ax = plt.subplots(num="BHR71 setting {} spw {}".format(setting, spw), clear=True)
i = ax.step(
    spectrum_imager.spectral_axis.to(
        u.GHz, equivalencies=u.doppler_radio(restfreq_imager)
    ).value,
    spectrum_imager.to(u.K).value,
    label="Imager",
)
if erda:
    c = ax.step(
        spectrum_casa.spectral_axis.to(
            u.GHz, equivalencies=u.doppler_radio(restfreq_casa)
        ).value,
        spectrum_casa.to(u.K).value,
        label="Casa",
    )
baseline = np.zeros_like(spectrum_imager.spectral_axis.value)
b = ax.plot(
    spectrum_imager.spectral_axis.to(
        u.GHz, equivalencies=u.doppler_radio(restfreq_imager)
    ).value,
    baseline,
    color="black",
    linestyle="dotted",
)
ax.set_xlabel("Frequency (GHz)")
ax.set_ylabel("Brightness temperature (K)")
ll = ax.legend()